In [1]:
from tqdm import tqdm
import pandas_datareader as web
import pandas as pd
import datetime as dt
import numpy as np
from scipy.optimize import minimize
import pyfolio as pf
import warnings
import yfinance as yf
import matplotlib.pyplot as plt
import empyrical
from ta import momentum 
from ta import trend
from ta import volatility
from ta import volume
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from dateutil.relativedelta import relativedelta 
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import objective_functions
import investpy
from datetime import datetime, timedelta, date
from arch import arch_model
plt.style.use('seaborn')
print(datetime.now())

from alpha_vantage.timeseries import TimeSeries
api_alpha_vantage = 'TVVU975Z2WVV7791'
ts = TimeSeries(key = api_alpha_vantage, output_format= 'pandas')
import time
import seaborn as sns

from hurst import compute_Hc, random_walk
hurst = lambda x: compute_Hc(x)[0]

import math
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

C:\Users\felip\Anaconda3\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


2023-08-30 14:38:10.051900


<ipython-input-1-20034bd790ac>:33: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [2]:
warnings.filterwarnings('ignore')
yf.pdr_override()

#%% Inputs

#Restrições
bound = (0.0, .3)
rent_alvo = np.arange(-1.00, 1, 0.01)

#Data
first_price_date = dt.datetime(2014, 1, 1)
if date.today().day > 27: #No final do mês ele considera o mês como finalizado
    last_price_date = dt.datetime(2023, date.today().month + 1, 1)
else:
    last_price_date = dt.datetime(2023, date.today().month, 1)


symbols = ['PETR4.SA', 'AMER3.SA', 'ITUB4.SA','VALE3.SA','WEGE3.SA','ELET3.SA','B3SA3.SA',\
           'VIIA3.SA','COGN3.SA','OIBR3.SA','CIEL3.SA','SHOW3.SA','EMBR3.SA','CVCB3.SA',\
           'ALPA4.SA','ABEV3.SA', 'BPAN4.SA','BBSE3.SA','BBDC4.SA','BBAS3.SA',\
           'BRKM5.SA','BRFS3.SA','CCRO3.SA','CMIG4.SA','CPLE6.SA','CSAN3.SA','CPFE3.SA','CYRE3.SA',\
           'ECOR3.SA','ENEV3.SA','EGIE3.SA','EQTL3.SA','EZTC3.SA','FLRY3.SA',\
           'GGBR4.SA','GOLL4.SA','HYPE3.SA','JBSS3.SA','JHSF3.SA',\
           'KLBN11.SA',\
           'LREN3.SA','MGLU3.SA','MRFG3.SA','BEEF3.SA','MRVE3.SA','MULT3.SA','PRIO3.SA','QUAL3.SA',\
           'RADL3.SA',\
           #'RAIL3.SA',\
           'SBSP3.SA','SANB11.SA','SLCE3.SA','TAEE11.SA','VIVT3.SA',\
           'TIMS3.SA','TOTS3.SA','USIM5.SA','YDUQ3.SA']

benchmarks = ['^BVSP']

preselection = 'yes'

ml_method = 'RF'

opt_method = '1/N'

trainning_test_size = 'yearly'

test_size = 'monthly'

data_source = 'yf'

In [3]:
#%% Download
data = pd.DataFrame()
data_high = pd.DataFrame()
data_low = pd.DataFrame()
data_volume = pd.DataFrame()
data_benchmark = pd.DataFrame()

#Baixando os dados
#Várias formas de puxar os dados 
if data_source == 'excel':
    data =\
    pd.read_excel('data.xlsx').set_index('date').fillna(method='bfill').loc[first_price_date:]
    data_high = \
    pd.read_excel('data_high.xlsx').set_index('date').fillna(method='bfill').loc[first_price_date:]
    data_low =\
    pd.read_excel('data_low.xlsx').set_index('date').fillna(method='bfill').loc[first_price_date:]
    data_volume =\
    pd.read_excel('data_volume.xlsx').set_index('date').fillna(method='bfill').loc[first_price_date:]
else:
    for sym in tqdm(symbols):
        if data_source == 'yf':
            if sym[-3:]!='.SA':
                if sym=='BM&FBOVESPA Real Estate IFIX':
                    dados=investpy.get_index_historical_data(sym, country='brazil',\
                    from_date=first_price_date.strftime('%d/%m/%Y'),\
                    to_date=last_price_date.strftime('%d/%m/%Y')).rename(columns={'Close':'Adj Close'})
                elif sym=='Fundo de Invest Ishares SP 500':
                    dados=investpy.get_etf_historical_data(sym, country='brazil',\
                    from_date=first_price_date.strftime('%d/%m/%Y'),\
                    to_date=last_price_date.strftime('%d/%m/%Y')).rename(columns={'Close':'Adj Close'})
            else:
                dados=web.data.get_data_yahoo(sym,first_price_date,last_price_date)
            adj = dados['Adj Close']/dados['Close'] #Ajustar dados
            
        elif data_source == 'alpha':
            try:
                dados, meta_dados = ts.get_daily_adjusted(symbol = sym + 'O', outputsize = 'full')
                dados = dados.sort_index()
                dados.rename(columns={'1. open':'Open'}, inplace = True)
                dados.rename(columns={'2. high':'High'}, inplace = True)
                dados.rename(columns={'3. low':'Low'}, inplace = True)
                dados.rename(columns={'4. close':'Close'}, inplace = True)
                dados.rename(columns={'5. adjusted close':'Adj Close'}, inplace = True)
                dados.rename(columns={'6. volume':'Volume'}, inplace = True)
                dados.drop(['7. dividend amount', '8. split coefficient'], axis=1, inplace = True)
            except ValueError:
                time.sleep(60)
                dados, meta_dados = ts.get_daily_adjusted(symbol = sym + 'O', outputsize = 'full')
                dados = dados.sort_index()
                dados.rename(columns={'1. open':'Open'}, inplace = True)
                dados.rename(columns={'2. high':'High'}, inplace = True)
                dados.rename(columns={'3. low':'Low'}, inplace = True)
                dados.rename(columns={'4. close':'Close'}, inplace = True)
                dados.rename(columns={'5. adjusted close':'Adj Close'}, inplace = True)
                dados.rename(columns={'6. volume':'Volume'}, inplace = True)
                dados.drop(['7. dividend amount', '8. split coefficient'], axis=1, inplace = True)     
            adj = 1 #Dados já ajustados

        dados = dados.loc[first_price_date:last_price_date]
        data[sym] = dados['Adj Close']
        data_high[sym] = dados['High']*adj
        data_low[sym] = dados['Low']*adj
        data_volume[sym] = dados['Volume']

data_total = data
data = data.fillna(method = 'ffill')
data_high = data_high.fillna(method = 'ffill')
data_low = data_low.fillna(method = 'ffill')
data_volume = data_volume.fillna(method = 'ffill')

data_benchmark = pd.DataFrame()

returns = np.log(data / data.shift(1)) 
returns = returns.dropna()

#Baixando os benchmarks
for benchmark in benchmarks:
    data_benchmark[benchmark] = \
    web.data.get_data_yahoo(benchmark,first_price_date,last_price_date)['Adj Close']
data_benchmark = data_benchmark.dropna()

returns_benchmark = np.log(data_benchmark / data_benchmark.shift(1)) 
returns_benchmark = returns_benchmark.dropna()

  0%|          | 0/58 [00:00<?, ?it/s]

[*********************100%***********************]  1 of 1 completed

  2%|▏         | 1/58 [00:00<00:47,  1.21it/s]


[*********************100%***********************]  1 of 1 completed

  3%|▎         | 2/58 [00:01<00:42,  1.33it/s]

...


[*********************100%***********************]  1 of 1 completed

 98%|█████████▊| 57/58 [00:33<00:00,  1.83it/s]


[*********************100%***********************]  1 of 1 completed

100%|██████████| 58/58 [00:33<00:00,  1.72it/s]

[*********************100%***********************]  1 of 1 completed


In [4]:
#%% Optimization 

### Markowitz with preselection ###
#if preselection == 'yes':
    
def portfolio_variance_ps(w): #matriz de covariância
    _w = np.asarray(w) #tirar o "_"
    return _w.dot(covariance_matrix).dot(_w)

def portfolio_return_ps(w):
    _w = np.asarray(w)
    return _w.dot(annualized_returns)

def portfolio_variance_gradient_ps(w):
    _w = np.asarray(w)
    return 2 * _w.dot(covariance_matrix)

def minimize_portfolio_variance_ps(min_return):
    assets_number = len(winners_list)
    bnds = [bound for i in range(assets_number)] 
    initial_guess = [1 / assets_number for i in range(assets_number)]
    cons = ({'type': 'eq', 'fun': lambda w: w.sum() - 1},
            {'type': 'ineq', 'fun': lambda w: portfolio_return_ps(w) - min_return})
    return minimize(portfolio_variance_ps, initial_guess,
                    constraints=cons,
                    bounds=bnds,
                    options={'disp': False},
                    method='SLSQP',
                    jac=portfolio_variance_gradient_ps)

def calculate_mv_frontier_ps():
    frontier = pd.Series()
    weights = pd.DataFrame()
    for min_return in rent_alvo:
        result = minimize_portfolio_variance_ps(min_return)
        if not result.success:
            continue
        w_frontier = result.x
        port_sigma = np.sqrt(portfolio_variance_ps(w_frontier))
        port_return = portfolio_return_ps(w_frontier)
        frontier.at[port_sigma] = port_return

        for i, win in enumerate(winners_list):
            weights.at[round(port_sigma,2), win] = round(w_frontier[i],4)
            
    return frontier, weights

### Markowitz MV ###

def portfolio_variance(w):
    _w = np.asarray(w) #tirar o "_"
    return _w.dot(covariance_matrix).dot(_w)

def portfolio_return(w):
    _w = np.asarray(w)
    return _w.dot(annualized_returns)

def portfolio_variance_gradient(w):
    _w = np.asarray(w)
    return 2 * _w.dot(covariance_matrix)

def minimize_portfolio_variance(min_return):
    assets_number = len(symbols)
    bnds = [bound for i in range(assets_number)] 
    initial_guess = [1 / assets_number for i in range(assets_number)]
    cons = ({'type': 'eq', 'fun': lambda w: w.sum() - 1},
            {'type': 'ineq', 'fun': lambda w: portfolio_return(w) - min_return})
    return minimize(portfolio_variance, initial_guess,
                    constraints=cons,
                    bounds=bnds,
                    options={'disp': False},
                    method='SLSQP',
                    jac=portfolio_variance_gradient)

def calculate_mv_frontier():
    frontier = pd.Series()
    weights = pd.DataFrame()
    for min_return in rent_alvo:
        result = minimize_portfolio_variance_ps(min_return)
        if not result.success:
            continue
        w_frontier = result.x
        port_sigma = np.sqrt(portfolio_variance_ps(w_frontier))
        port_return = portfolio_return_ps(w_frontier)
        frontier.at[port_sigma] = port_return

        for i, symbol in enumerate(symbols):
            weights.at[round(port_sigma,2), symbol] = round(w_frontier[i],4)
            
    return frontier, weights


### Hierarchical Risk Parity (HRP) ###
def calculate_hrp(returns_escolhido):
    hrp = hierarchical_risk_parity.HRPOpt(returns_escolhido)
    a=hrp.hrp_portfolio()
    a=pd.Series(a)
    a = pd.DataFrame(a)
    weights = a.T
    return weights

### 1/N ###

def calculate_ns():
    weights = pd.DataFrame(columns=symbols, index = [0])
    for sym in symbols:
        weights[sym][0] = 1.0/len(symbols)  

    return weights   

#Calculando o retorno acumulado
def portfolio_performance(initial_data_slc, end_data_slc):    
    portfolio = weights.iloc[0]
    sliced_returns = returns.loc[initial_data_slc:end_data_slc]
    retorno_ponderado = (portfolio * sliced_returns)
    retorno = retorno_ponderado.sum(axis=1)
    retorno_acumulado = (retorno+1).cumprod()
            
    return retorno, retorno_acumulado, portfolio 

def garch(serie_dados, periodos):
    
    ativo_slice=serie_dados
    retorno=np.log(ativo_slice/ativo_slice.shift(1)).dropna()
    padronizado=(retorno-retorno.mean())/retorno.std()
    
    garch_model=arch_model(retorno, p=1, q=1, vol='GARCH', dist='Normal', rescale=False)
    resultados=garch_model.fit(disp='off')
    parametros=resultados.params
    
    gama=1-parametros['alpha[1]']-parametros['beta[1]']
    vol_incondicional=retorno.std()*(252**.5)
    vol_longo_prazo=(252*parametros['omega']/gama)**.5
    
    serie_garch=pd.Series(index=retorno.index)
    serie_garch.iloc[0]=retorno.iloc[0]**2
    for i in retorno.index[1:]:
        #print(i)
        serie_garch.loc[i]=parametros['omega']+parametros['alpha[1]']*(retorno.shift(1).loc[i]**2)+\
        parametros['beta[1]']*serie_garch.shift(1).loc[i]
            
    return ((resultados.forecast(horizon=periodos, reindex=False).variance.iloc[0])*252)**.5,\
           (serie_garch*252)**.5

def portfolio_results(ativos, pesos, data_ini):
    
    returns_mes=pd.DataFrame()

    for i in ativos:
        returns_mes[i]=\
        web.data.get_data_yahoo(i,data_ini,datetime(2099,12,31).date())['Adj Close']

    returns_mes=(returns_mes/returns_mes.shift(1)).fillna(1)
    returns_portfolio=np.dot(returns_mes.cumprod(),pesos)

    benchmark_mes=\
    web.data.get_data_yahoo(benchmark,data_ini,datetime(2099,12,31).date())['Adj Close']
    ret_benchmark_mes=(benchmark_mes/benchmark_mes.shift(1)).fillna(1).cumprod()

    comparative=pd.DataFrame(index=ret_benchmark_mes.index)
    comparative.loc[:,'Strategy']=returns_portfolio
    comparative.loc[:,'Benchmark']=ret_benchmark_mes.values

    return comparative

def prints_returns():
    print('Strategy')
    print('Retorno: ',\
    round(np.log(returns_portfolio/returns_portfolio.shift(1))['Strategy'].dropna().mean()*252*100,2))
    print('Vol: ',\
    round(np.log(returns_portfolio/returns_portfolio.shift(1))['Strategy'].dropna().std()*(252**.5)*100,2))
    print('------------------')
    print('Benchmark')
    print('Retorno:',\
    round(np.log(returns_portfolio/returns_portfolio.shift(1))['Benchmark'].dropna().mean()*252*100,2))
    print('Vol:',\
    round(np.log(returns_portfolio/returns_portfolio.shift(1))['Benchmark'].dropna().std()*(252**.5),2))
    
def normalizar(df):
    return df/df.iloc[0]

def cf_analysis(cm):
    
    t=cm.iloc[0,0]+cm.iloc[0,1]+cm.iloc[1,0]+cm.iloc[1,1]
    acc_pos=cm.iloc[0,0]/(cm.iloc[0,0]+cm.iloc[0,1])
    tot_pos=(cm.iloc[0,0]+cm.iloc[1,1])/t
    acc_neg=cm.iloc[1,0]/(cm.iloc[1,0]+cm.iloc[1,1])
    tot_neg=(cm.iloc[0,1]+cm.iloc[1,0])/t
    acc_geral =(cm.iloc[0,0]+cm.iloc[1,0])/t
    
    print('')
    print('Acc Positive:',round(acc_pos,3),'x Tot Positive:',round(tot_pos,3))
    print('Acc Negative:',round(acc_neg,3),'x Tot Negative:',round(tot_neg,3))
    print('Acc Geral:',round(acc_geral,3))
    
    return 0

def MAPE(y_true, y_predict): 
    y_true, y_predict = np.array(y_true), np.array(y_predict)
    return np.mean(np.abs((y_true - y_predict) / y_true)) * 100

def analise_rrg(serie, n_days):
    #ratio entre as ações
    ratio=(100*serie)
    #macd do ratio
    macd=trend.MACD(ratio,30,10,9).macd()
    #eixo x do rrg
    eixo_x=100*(macd/ratio)
    #ema do eixo_x
    ema=trend.EMAIndicator(eixo_x, 9).ema_indicator()
    #eixo y do rrg
    eixo_y=eixo_x-ema
    
    posicao=(eixo_x,eixo_y)
    posicao_past=(eixo_x.shift(n_days), eixo_y.shift(n_days))
           
    dx = eixo_x - eixo_x.shift(n_days)
    dy = eixo_y - eixo_y.shift(n_days)
    
    posicao_pred = (eixo_x + dx, eixo_y + dy)
            
    intensidade=(dx**2+dy**2)**.5
    
    return posicao, posicao_past, posicao_pred, intensidade

In [5]:
data = data.fillna(method = 'bfill')
data_high = data_high.fillna(method = 'bfill')
data_low = data_low.fillna(method = 'bfill')
data_volume = data_volume.fillna(method = 'bfill')

In [6]:
#%% Run

#Returns Series
strategy_return = pd.Series()
bench_return = pd.Series()
naive_return = pd.Series()
looser_return = pd.Series()

#Listas para avaliar acurácia e criar matriz de confusão
predicted_values = []
actual_values = []
total_confusion_matrix = pd.DataFrame(0, index=['P','N'], columns=['T','F'])

#Listas para calcular indicadores de precisão das previsões
mse=[]
mape=[]
r_mean=[]

#Rankeando as winners/loosers para compará-las
selected_rank_winners = []
selected_rank_loosers = []

#Criando incremento das iterações
if test_size == 'monthly':
    increment = relativedelta(months=1)
elif test_size == 'quarterly':
    increment = relativedelta(months=3)
elif test_size == 'semesterly':
    increment = relativedelta(months=6)
elif test_size == 'yearly':
    increment = relativedelta(months=12)

#Criando incremento para determinar base de treino inicial
if trainning_test_size == 'monthly':
    increment_train = relativedelta(months=0)
elif trainning_test_size == 'quarterly':
    increment_train = relativedelta(months=2)
elif trainning_test_size == 'semesterly':
    increment_train = relativedelta(months=5)
elif trainning_test_size == 'yearly':
    increment_train = relativedelta(months=11)

#Data inicial do modelo
initial_data = first_price_date
#Iniciar variável da data limite de treino 
dat_aux = first_price_date + increment_train #auxiliar para iniciar o end_data
end_data = \
returns[(returns.index.year == dat_aux.year) &\
        (returns.index.month == dat_aux.month)].index[-1]
    
while end_data + increment <  last_price_date:
    
    if len(strategy_return) == 0: #se for a primeira iteração entra aqui
        #data final de teste
        dat_aux = first_price_date + increment_train + increment
        end_data = returns[(returns.index.year==dat_aux.year) &\
                           (returns.index.month==dat_aux.month)].index[-1]
        #data final treino
        dat_aux = end_data - increment
        limit_train_data=\
        returns[(returns.index.year == dat_aux.year) &\
                (returns.index.month == dat_aux.month)].index[-1]
        
    else:
        #Nova data limite de treino
        limit_train_data = end_data
        #Nova data inicial
        dat_aux = initial_data + increment
        initial_data=returns[(returns.index.year == dat_aux.year) &\
                             (returns.index.month == dat_aux.month)].index[0] 
        #Nova data limite de teste
        dat_aux = end_data + increment
        end_data = returns[(returns.index.year == dat_aux.year) &\
                           (returns.index.month==dat_aux.month)].index[-1]
            
    #Determinando o tamanho da base de teste (tamanho total - tamanho de treino)
    size_test = len(returns.loc[initial_data:end_data])-\
                len(returns.loc[initial_data:limit_train_data])
    
    #Determinando o início da base de teste
    initial_test_data = returns.loc[initial_data:end_data].iloc[-size_test:].index[0]
        
    #Printar datas chave
    print('------------------------------------------------------')
    print('Início Treino: ', initial_data)
    print('Fim Treino:    ', limit_train_data)
    print('Início Teste:  ', initial_test_data)
    print('Fim Teste:     ', end_data)
        
    #Dataframe que consolida valores previstos
    prediction_consolidate = pd.DataFrame(columns = symbols)
        
    #Criar Matriz de Confusão
    confusion_matrix=pd.DataFrame(0, index=['P','N'], columns=['T','F'])
    
    for sym in symbols:
        #Slicing Database
        all_data = normalizar(data.loc[initial_data-increment:end_data])
        all_data_high = normalizar(data_high.loc[initial_data-increment:end_data])
        all_data_low = normalizar(data_low.loc[initial_data-increment:end_data])
        all_data_volume = data_volume.loc[initial_data-increment:end_data]
            
        #Creating the features dataframe
        df = pd.DataFrame()
        df['RSI'] = momentum.RSIIndicator(close=all_data[sym]).rsi()
        df['ATR'] = volatility.AverageTrueRange(high=all_data_high[sym], low=all_data_low[sym],\
                                                close=all_data[sym]).average_true_range()
        df['OBV'] = volume.on_balance_volume(close=all_data[sym], volume=all_data_volume[sym])
        df['PSAR'] = trend.PSARIndicator(high=all_data_high[sym], low=all_data_low[sym],\
                                         close=all_data[sym]).psar()
            
        df['EWMA 5 P'] = pd.Series.ewm(all_data[sym], span=5).mean()
        df['EWMA 10 P'] = pd.Series.ewm(all_data[sym], span=10).mean()
        df['EWMA 22 P'] = pd.Series.ewm(all_data[sym], span=22).mean()
        df['EWMA 66 P'] = pd.Series.ewm(all_data[sym], span=66).mean()
            
        df['EWMA 5 V'] = pd.Series.ewm(all_data_volume[sym], span=5).mean()
        df['EWMA 10 V'] = pd.Series.ewm(all_data_volume[sym], span=10).mean()
        df['EWMA 22 V'] = pd.Series.ewm(all_data_volume[sym], span=22).mean()
        df['EWMA 66 V'] = pd.Series.ewm(all_data_volume[sym], span=66).mean()  
            
        #Adding simple moving average for price
        df['SMA 5 P'] = all_data[sym].rolling(5).mean()
        df['SMA 10 P'] = all_data[sym].rolling(10).mean()
        df['SMA 22 P'] = all_data[sym].rolling(22).mean()
        df['SMA 66 P'] = all_data[sym].rolling(66).mean()
            
        #Adding simple moving average for volume
        df['SMA 5 V'] = all_data_volume[sym].rolling(5).mean()
        df['SMA 10 V'] = all_data_volume[sym].rolling(10).mean()
        df['SMA 22 V'] = all_data_volume[sym].rolling(22).mean()
        df['SMA 66 V'] = all_data_volume[sym].rolling(66).mean()
            
        #Adding Momentum
        df['Momentum 5'] = all_data[sym] - all_data[sym].shift(5)
        df['Momentum 10'] = all_data[sym] - all_data[sym].shift(10)
        df['Momentum 22'] = all_data[sym] - all_data[sym].shift(22)
        df['Momentum 66'] = all_data[sym] - all_data[sym].shift(66)
            
        #Adding Rolling Std Dev of prices
        df['StdDev 5 Ret'] = all_data[sym].pct_change(1).rolling(5).std()
        df['StdDev 10 Ret'] = all_data[sym].pct_change(1).rolling(10).std()
        df['StdDev 22 Ret'] = all_data[sym].pct_change(1).rolling(22).std()
        df['StdDev 66 Ret'] = all_data[sym].pct_change(1).rolling(66).std()
            
        df['Bollinger_High_2'] = \
        volatility.BollingerBands(close = all_data[sym], window = 20, window_dev = 2)\
        .bollinger_hband() 
    
        df['Bollinger_Low_2'] = \
        volatility.BollingerBands(close = all_data[sym], window = 20, window_dev = 2)\
        .bollinger_lband() 
    
        df['Bollinger_High_1'] = \
        volatility.BollingerBands(close = all_data[sym], window = 20, window_dev = 1)\
        .bollinger_hband() 

        df['Bollinger_Low_1'] = \
        volatility.BollingerBands(close = all_data[sym], window = 20, window_dev = 1)\
        .bollinger_lband()
            
        df['MACD'] = \
        trend.MACD(close = all_data[sym], window_slow = 26, window_fast = 12, window_sign = 9)\
        .macd() 
            
        df['MACD_Signal'] = \
        trend.MACD(close = all_data[sym], window_slow = 26, window_fast = 12, window_sign = 9)\
        .macd_signal() 
            
        df['MACD_Hist'] = \
        trend.MACD(close = all_data[sym], window_slow = 26, window_fast = 12, window_sign = 9)\
        .macd_diff() 
            
        df['Ultimate Oscillator']=\
        momentum.UltimateOscillator(high = all_data_high[sym], low = all_data_low[sym],\
                                    close = all_data[sym]).ultimate_oscillator() 
            
        df['Williams']=\
        momentum.WilliamsRIndicator(high = all_data_high[sym], low = all_data_low[sym],\
        close = all_data[sym]).williams_r().replace(np.inf, 100).replace(-np.inf, -100)
            
        #Adding RRG
        df['RRG 5 X'] = analise_rrg(all_data[sym], 5)[0][0]
        df['RRG 10 X'] = analise_rrg(all_data[sym], 10)[0][0]
        df['RRG 22 X'] = analise_rrg(all_data[sym], 22)[0][0]
            
        df['RRG 5 Y'] = analise_rrg(all_data[sym], 5)[0][1]
        df['RRG 10 Y'] = analise_rrg(all_data[sym], 10)[0][1]
        df['RRG 22 Y'] = analise_rrg(all_data[sym], 22)[0][1]
            
        df['RRG 5 X Pred'] = analise_rrg(all_data[sym], 5)[2][0]
        df['RRG 10 X Pred'] = analise_rrg(all_data[sym], 10)[2][0]
        df['RRG 22 X Pred'] = analise_rrg(all_data[sym], 22)[2][0]
            
        df['RRG 5 Y Pred'] = analise_rrg(all_data[sym], 5)[2][1]
        df['RRG 10 Y Pred'] = analise_rrg(all_data[sym], 10)[2][1]
        df['RRG 22 Y Pred'] = analise_rrg(all_data[sym], 22)[2][1]
            
        df['RRG 5 Intensidade'] = analise_rrg(all_data[sym], 5)[3]
        df['RRG 10 Intensidade'] = analise_rrg(all_data[sym], 10)[3]
        df['RRG 22 Intensidade'] = analise_rrg(all_data[sym], 22)[3]
         
            
        df = df.loc[initial_data:end_data] #Slice final
        df_total = df #Salvar DataFrame 
        df = df.dropna() 
        
        #Retornos Diários
        retornos = np.log(all_data[sym]/all_data[sym].shift(1))  
        retornos = retornos.dropna()
        
        #Retornos após período de size_test dias
        retornos_periodo = np.log(all_data[sym]/all_data[sym].shift(size_test))
        retornos_periodo = retornos_periodo.dropna()
        
        #Criar modelo de normalização
        scaler = StandardScaler()
        scaler.fit(df.shift(size_test).dropna())
        
        #Criar Dataframe normalizado
        df_normalizado = scaler.transform(df.shift(size_test).dropna())
        df_normalizado = \
        pd.DataFrame(df_normalizado, index = df.shift(size_test).dropna().index, columns = df.columns)
            
        #Cria Dataframe com valores binários
        #se após um período size_test dias subiu, 1, se não, -1
        df_trend_deterministic = \
        pd.DataFrame(index = df_normalizado.index, columns = df_normalizado.columns)
        for col in df_trend_deterministic.columns:
            df_trend_deterministic.loc[:,col] = \
            np.where(df_normalizado[col].pct_change(size_test)>0, 1,-1)
            
        #Cria X_train, X_test, y_train, y_test
        X = df_trend_deterministic
        y_reg = pd.merge(df, retornos_periodo, how='inner', on = 'Date')[sym].iloc[size_test:]
        y_bin = np.where(y_reg > 0 , 1, -1)
        y_df = y_reg.copy()
        y = pd.DataFrame(y_bin, index = y_reg.index).iloc[:,0]
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size_test, shuffle=False)
            
        #Determinar modelo ML
        if ml_method == 'KNN':
            model = KNeighborsClassifier(n_neighbors=5)
        if ml_method == 'RF':
            model = RandomForestClassifier(n_estimators = 100, random_state = 0)
        if ml_method == 'SVR':
            try:
                model = make_pipeline(StandardScaler(), SVC(gamma='auto'))
            except ValueError:
                model = RandomForestClassifier(n_estimators = 100, random_state = 0)
        
        #Trienar ML e prever resultados
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        #Guardar MSE, MAPE e previsão média
        mse.append(mean_squared_error(y_test+1, y_pred+1))
        mape.append(MAPE(y_test+1, y_pred+1))
        r_mean.append(y_pred.mean())

        #Registrando a previsão 
        prediction = pd.DataFrame(data=y_pred,columns=[sym])
        prediction_consolidate[sym] = prediction[sym]
            
        #creating confusion matrix     
        if prediction[sym].sum() > 0:
            if (1+retornos.loc[initial_test_data:]).cumprod().iloc[-1] > 1:
                confusion_matrix.loc['P','T'] = confusion_matrix.loc['P','T'] + 1
            else:
                confusion_matrix.loc['P','F'] = confusion_matrix.loc['P','F'] + 1
        elif prediction[sym].sum() < 1:
            if (1+retornos.loc[initial_test_data:]).cumprod().iloc[-1] < 1:
                confusion_matrix.loc['N','T'] = confusion_matrix.loc['N','T'] + 1
            else:
                confusion_matrix.loc['N','F'] = confusion_matrix.loc['N','F'] + 1
        
        #Guardar valores previstos e realizados
        predicted_values.append((prediction[sym]).iloc[-1])
        actual_values.append((retornos.loc[initial_test_data:]).cumprod().iloc[-1])
                    
    #Adicionar dados da matriz de confusão á amtriz consolidada
    total_confusion_matrix = total_confusion_matrix + confusion_matrix
            
    #Calculando o retorno acumulado
    cummulative_ret_pred = prediction_consolidate.sum()
        
    #Selecionando os ativos com retorno positivo
    winners_df = pd.DataFrame(columns=['Rent'])
    winners_df['Rent'] = cummulative_ret_pred
    total_df = winners_df.copy() #Salvar winners_df original para fazer loosers
    winners_df = winners_df.nlargest(10,'Rent')
    winners_list = list(winners_df.index)
    
    #Criar inputs da otimização
    winners_ret = returns.filter(items=winners_list, axis=1)
    winners_ret =  winners_ret.loc[y_train.index[0]:y_train.index[-1]]  
    covariance_matrix = np.asarray(252 * winners_ret.cov())
    annualized_returns = np.asarray(252 * winners_ret.mean())
        
    #Criando ranking dos retornos para ver se as escolhidas foram bem no relativo 
    ranking =\
    all_data.pct_change(size_test).iloc[-1].rank(ascending = False).loc[winners_df.index]
    selected_rank_winners.append((ranking-.5).mean())
                
    #Ponderação das carteiras       
    if preselection == 'yes':
        if opt_method == 'MV':
            frontier, weights = calculate_mv_frontier_ps()
        if opt_method == 'HRP':
            weights = calculate_hrp(winners_ret)
        if opt_method == '1/N':    
            winners_list = list(winners_ret.columns)
            weights = pd.DataFrame(columns = winners_list)
            weights.loc[0] = [1 / len(winners_list) for i in range(len(winners_list))]
    else:
        if opt_method == 'MV':
            frontier, weights = calculate_mv_frontier()
        if opt_method == 'HRP':
            weights = calculate_hrp(returns)
        if opt_method == '1/N':
            weights = calculate_ns()
        
    #Resultados da Carteira Winners
    retorno, retorno_acumulado, portfolio = portfolio_performance(initial_test_data, end_data)
    strategy_return = pd.concat([strategy_return, retorno])
        
    print('')
    print('Portfolio:')
    print(portfolio)
    print('')
    
    #Resultados do Benchmark
    benchmark_rets = returns_benchmark.loc[initial_data:end_data]
    bench_return = pd.concat([bench_return, benchmark_rets.loc[initial_test_data:end_data].squeeze()])
        
    #Criando carteira Naive
    winners_list = list(all_data.columns)
    weights = pd.DataFrame(columns = winners_list)
    weights.loc[0] = [1 / len(winners_list) for i in range(len(winners_list))]
       
    #Resultados da Carteira Naive
    retorno_naive, retorno_acumulado_naive, portfolio_naive = \
    portfolio_performance(initial_test_data,end_data)
    naive_return = pd.concat([naive_return, retorno_naive])
        
    #Criando carteira Looser
    winners_df = total_df.nsmallest(10,'Rent')
    winners_list = list(winners_df.index)    
    winners_ret = returns.filter(items=winners_list, axis=1)
    winners_ret =  winners_ret.loc[y_train.index[0]:y_train.index[-1]]
    winners_list = list(winners_ret.columns)
    weights = pd.DataFrame(columns = winners_list)
    weights.loc[0] = [1 / len(winners_list) for i in range(len(winners_list))]
    
    #Resultados da Carteira Looser
    retorno_looser, retorno_acumulado_looser, portfolio_looser =\
    portfolio_performance(y_test.index[0],y_test.index[-1])
    looser_return = pd.concat([looser_return, retorno_looser])
    
    #Criando ranking dos retornos para ver se as escolhidas foram bem no relativo
    ranking = \
    all_data.pct_change(size_test).iloc[-1].rank(ascending = False).loc[winners_df.index]
    selected_rank_loosers.append((ranking-.5).mean())
        
    #Prints e mais prints
    print('Janela Treino:         ', \
    len(returns.loc[initial_data:end_data].loc[initial_data:limit_train_data])-\
    len(returns.loc[initial_data:end_data].loc[initial_test_data:end_data]))
    print('Janela Teste:        ', \
    len(returns.loc[initial_data:end_data].loc[initial_test_data:end_data]))
    print('Retorno:              ', (retorno+1).cumprod()[-1]-1)
    print('Retorno Looser:              ', (retorno_looser+1).cumprod()[-1]-1)
    print('Retorno Benchmark:    ', \
    (benchmark_rets.loc[initial_test_data:end_data]+1).cumprod().iloc[-1].values[0]-1)
    print('Retornos Acumulados:  ',\
    (strategy_return+1).cumprod()[-1],'x',(bench_return+1).cumprod().iloc[-1], 'x',\
    (naive_return+1).cumprod().iloc[-1], 'x',(looser_return+1).cumprod().iloc[-1] )
    print('Volatilidade:         ', np.sqrt(portfolio_variance(portfolio)))
    print('Sharpe Ratio:         ', empyrical.sharpe_ratio(returns=retorno))
    print('Maximum Draw Down:    ', empyrical.max_drawdown(returns=retorno))
    print('Value at Risk:        ', empyrical.value_at_risk(returns=retorno, cutoff=0.01))
    #print('Beta:                 ',\
    #      empyrical.beta(returns=retorno, factor_returns=benchmark_rets[benchmarks])) 
    #print('------------------------------------------------------')
    print('')
    print(confusion_matrix)
    cf_analysis(confusion_matrix)
    print('')
    print(total_confusion_matrix)
    cf_analysis(total_confusion_matrix)
    print('')
    print('MSE:',round(100*sum(mse)/len(mse),3),'%')
    print('MAPE:',round(sum(mape)/len(mape),3),'%')
    print('Retorno Médio:',round(100*sum(r_mean)/len(r_mean),3),'%')
    print('Ranking Winners:',round(sum(selected_rank_winners)/len(selected_rank_winners),3))
    print('Ranking Loosers:',round(sum(selected_rank_loosers)/len(selected_rank_loosers),3))
    print('')

        
#criando incremento para nova data inicial
dat_aux = initial_data + increment
new_initial_data = returns[(returns.index.year == dat_aux.year) &\
                           (returns.index.month == dat_aux.month)].index[0] 

------------------------------------------------------
Início Treino:  2014-01-01 00:00:00
Fim Treino:     2014-12-30 00:00:00
Início Teste:   2015-01-02 00:00:00
Fim Teste:      2015-01-30 00:00:00

Portfolio:
WEGE3.SA     0.1
EMBR3.SA     0.1
KLBN11.SA    0.1
BEEF3.SA     0.1
B3SA3.SA     0.1
BBAS3.SA     0.1
BRFS3.SA     0.1
TAEE11.SA    0.1
AMER3.SA     0.1
MRVE3.SA     0.1
Name: 0, dtype: float64

Janela Treino:          211
Janela Teste:         21
Retorno:               -0.03650795370632187
Retorno Looser:               -0.096609750371622
Retorno Benchmark:     -0.06481559819514082
Retornos Acumulados:   0.9634920462936781 x 0.9351844018048592 x 0.908432460932026 x 0.903390249628378
Volatilidade:          0.19688189380251261
Sharpe Ratio:          -2.354730793744612
Maximum Draw Down:     -0.053841123821848955
Value at Risk:         -0.020441616091287314

    T   F
P   8  17
N  26   7

Acc Positive: 0.32 x Tot Positive: 0.259
Acc Negative: 0.788 x Tot Negative: 0.741
Acc Geral: 


Portfolio:
VIIA3.SA     0.1
BRFS3.SA     0.1
EQTL3.SA     0.1
KLBN11.SA    0.1
LREN3.SA     0.1
RADL3.SA     0.1
TAEE11.SA    0.1
USIM5.SA     0.1
FLRY3.SA     0.1
SANB11.SA    0.1
Name: 0, dtype: float64

Janela Treino:          228
Janela Teste:         21
Retorno:               -0.08272928837640248
Retorno Looser:               -0.12988614043402524
Retorno Benchmark:     -0.08615905459588802
Retornos Acumulados:   0.9196465057470415 x 0.917861335621052 x 0.836598621901979 x 0.6992941121174737
Volatilidade:          0.15622430063565695
Sharpe Ratio:          -4.9815820431441065
Maximum Draw Down:     -0.11275651402032282
Value at Risk:         -0.026425546237695694

    T   F
P   2  23
N  29   4

Acc Positive: 0.08 x Tot Positive: 0.103
Acc Negative: 0.879 x Tot Negative: 0.897
Acc Geral: 0.534

     T    F
P   79   86
N  174  125

Acc Positive: 0.479 x Tot Positive: 0.44
Acc Negative: 0.582 x Tot Negative: 0.56
Acc Geral: 0.545

MSE: 175.17 %
MAPE: nan %
Retorno Médio: -18.678 %
Ra


Portfolio:
ITUB4.SA     0.1
KLBN11.SA    0.1
SBSP3.SA     0.1
BRKM5.SA     0.1
HYPE3.SA     0.1
RADL3.SA     0.1
CCRO3.SA     0.1
MRFG3.SA     0.1
EMBR3.SA     0.1
MRVE3.SA     0.1
Name: 0, dtype: float64

Janela Treino:          223
Janela Teste:         22
Retorno:               0.03576357203647862
Retorno Looser:               0.21796906119241521
Retorno Benchmark:     0.16199773460648048
Retornos Acumulados:   1.0176204313480748 x 0.9613007118250517 x 0.8773194373499617 x 0.6573823291575595
Volatilidade:          0.2309242089302191
Sharpe Ratio:          2.4034739200977877
Maximum Draw Down:     -0.03550967935203258
Value at Risk:         -0.02013672919414949

    T   F
P  15   5
N   6  32

Acc Positive: 0.75 x Tot Positive: 0.81
Acc Negative: 0.158 x Tot Negative: 0.19
Acc Geral: 0.362

     T    F
P  153  149
N  317  251

Acc Positive: 0.507 x Tot Positive: 0.464
Acc Negative: 0.558 x Tot Negative: 0.536
Acc Geral: 0.54

MSE: 173.87 %
MAPE: nan %
Retorno Médio: -21.888 %
Ranking


Portfolio:
PETR4.SA    0.1
VALE3.SA    0.1
WEGE3.SA    0.1
COGN3.SA    0.1
SHOW3.SA    0.1
CVCB3.SA    0.1
ABEV3.SA    0.1
CMIG4.SA    0.1
CPLE6.SA    0.1
CSAN3.SA    0.1
Name: 0, dtype: float64

Janela Treino:          228
Janela Teste:         20
Retorno:               0.08150287831948311
Retorno Looser:               0.08426356579273775
Retorno Benchmark:     0.11141044695444902
Retornos Acumulados:   1.2996536631481235 x 1.2290299988457534 x 1.1386714708346561 x 0.8290139976437526
Volatilidade:          0.29644958035743263
Sharpe Ratio:          6.884516479824838
Maximum Draw Down:     -0.013467217914548397
Value at Risk:         -0.011836267348691708

    T   F
P  35  12
N   2   9

Acc Positive: 0.745 x Tot Positive: 0.759
Acc Negative: 0.182 x Tot Negative: 0.241
Acc Geral: 0.638

     T    F
P  294  245
N  384  353

Acc Positive: 0.545 x Tot Positive: 0.507
Acc Negative: 0.521 x Tot Negative: 0.493
Acc Geral: 0.531

MSE: 170.027 %
MAPE: nan %
Retorno Médio: -8.779 %
Ranking Win


Portfolio:
CVCB3.SA    0.1
CPFE3.SA    0.1
CYRE3.SA    0.1
EQTL3.SA    0.1
EZTC3.SA    0.1
HYPE3.SA    0.1
MGLU3.SA    0.1
MRVE3.SA    0.1
MULT3.SA    0.1
PRIO3.SA    0.1
Name: 0, dtype: float64

Janela Treino:          227
Janela Teste:         22
Retorno:               -0.0728188322913953
Retorno Looser:               -0.024498956389985227
Retorno Benchmark:     -0.04705482712688869
Retornos Acumulados:   1.3875072087234683 x 1.1658430746027404 x 1.1723542859653073 x 1.0456291882781328
Volatilidade:          0.19519610882024818
Sharpe Ratio:          -1.8513897127190448
Maximum Draw Down:     -0.13454669124117705
Value at Risk:         -0.08935129759545972

    T   F
P  11  24
N  17   6

Acc Positive: 0.314 x Tot Positive: 0.293
Acc Negative: 0.739 x Tot Negative: 0.707
Acc Geral: 0.483

     T    F
P  413  378
N  458  433

Acc Positive: 0.522 x Tot Positive: 0.503
Acc Negative: 0.514 x Tot Negative: 0.497
Acc Geral: 0.518

MSE: 174.523 %
MAPE: nan %
Retorno Médio: -1.608 %
Ranking 

...


Portfolio:
OIBR3.SA     0.1
ALPA4.SA     0.1
BBAS3.SA     0.1
CSAN3.SA     0.1
GGBR4.SA     0.1
MGLU3.SA     0.1
PRIO3.SA     0.1
TAEE11.SA    0.1
TOTS3.SA     0.1
VALE3.SA     0.1
Name: 0, dtype: float64

Janela Treino:          228
Janela Teste:         19
Retorno:               0.1611600746948625
Retorno Looser:               0.1610420180067016
Retorno Benchmark:     0.1560137842807059
Retornos Acumulados:   4.081545329054586 x 1.7421165632837239 x 1.899969563383941 x 0.9255753579375527
Volatilidade:          0.5745489892112248
Sharpe Ratio:          8.47690356405067
Maximum Draw Down:     -0.02769535786520109
Value at Risk:         -0.019729599909648864

    T   F
P  29   5
N   3  21

Acc Positive: 0.853 x Tot Positive: 0.862
Acc Negative: 0.125 x Tot Negative: 0.138
Acc Geral: 0.552

      T     F
P  1272  1016
N   902   928

Acc Positive: 0.556 x Tot Positive: 0.534
Acc Negative: 0.493 x Tot Negative: 0.466
Acc Geral: 0.528

MSE: 171.299 %
MAPE: nan %
Retorno Médio: 11.811 %
Ran


Portfolio:
VALE3.SA    0.1
ELET3.SA    0.1
SHOW3.SA    0.1
EMBR3.SA    0.1
ABEV3.SA    0.1
BPAN4.SA    0.1
GGBR4.SA    0.1
JBSS3.SA    0.1
SLCE3.SA    0.1
VIVT3.SA    0.1
Name: 0, dtype: float64

Janela Treino:          226
Janela Teste:         21
Retorno:               -0.000755427674073883
Retorno Looser:               0.004682901778133353
Retorno Benchmark:     0.0040751590949985594
Retornos Acumulados:   4.982330342718952 x 2.0068882494667277 x 2.1499469234320965 x 1.0909970316740016
Volatilidade:          0.2189563292055518
Sharpe Ratio:          0.028383281303134833
Maximum Draw Down:     -0.0560573139027291
Value at Risk:         -0.02221800531780385

    T   F
P  13  28
N  13   4

Acc Positive: 0.317 x Tot Positive: 0.293
Acc Negative: 0.765 x Tot Negative: 0.707
Acc Geral: 0.448

      T     F
P  1398  1138
N   975  1013

Acc Positive: 0.551 x Tot Positive: 0.533
Acc Negative: 0.49 x Tot Negative: 0.467
Acc Geral: 0.525

MSE: 171.434 %
MAPE: nan %
Retorno Médio: 12.097 %
Ran


Portfolio:
JBSS3.SA    0.1
MRFG3.SA    0.1
PETR4.SA    0.1
CPLE6.SA    0.1
SBSP3.SA    0.1
EMBR3.SA    0.1
CSAN3.SA    0.1
WEGE3.SA    0.1
BRFS3.SA    0.1
CMIG4.SA    0.1
Name: 0, dtype: float64

Janela Treino:          226
Janela Teste:         21
Retorno:               -0.0031598755535222622
Retorno Looser:               0.0434118104927097
Retorno Benchmark:     0.07061134376510436
Retornos Acumulados:   4.520223178506796 x 1.7551991580253186 x 1.79005695355391 x 0.7870383805084666
Volatilidade:          0.20179864395886965
Sharpe Ratio:          -0.13749123642923572
Maximum Draw Down:     -0.05796215649382347
Value at Risk:         -0.025711004916080873

    T   F
P  10   8
N  10  30

Acc Positive: 0.556 x Tot Positive: 0.69
Acc Negative: 0.25 x Tot Negative: 0.31
Acc Geral: 0.345

      T     F
P  1461  1247
N  1123  1099

Acc Positive: 0.54 x Tot Positive: 0.519
Acc Negative: 0.505 x Tot Negative: 0.481
Acc Geral: 0.524

MSE: 170.991 %
MAPE: nan %
Retorno Médio: 9.879 %
Ranking W


Portfolio:
ELET3.SA     0.1
COGN3.SA     0.1
BRFS3.SA     0.1
CPFE3.SA     0.1
HYPE3.SA     0.1
CIEL3.SA     0.1
BEEF3.SA     0.1
TAEE11.SA    0.1
SHOW3.SA     0.1
CMIG4.SA     0.1
Name: 0, dtype: float64

Janela Treino:          226
Janela Teste:         23
Retorno:               0.11507626696182971
Retorno Looser:               0.026574158525022318
Retorno Benchmark:     0.059809023375261505
Retornos Acumulados:   5.060919027315646 x 1.6931553362276135 x 1.6090929607022433 x 0.6212115058068285
Volatilidade:          0.2492043448303651
Sharpe Ratio:          6.219952421673774
Maximum Draw Down:     -0.025300335198353615
Value at Risk:         -0.019093875629000074

    T   F
P  24   4
N  12  18

Acc Positive: 0.857 x Tot Positive: 0.724
Acc Negative: 0.4 x Tot Negative: 0.276
Acc Geral: 0.621

      T     F
P  1547  1332
N  1246  1211

Acc Positive: 0.537 x Tot Positive: 0.517
Acc Negative: 0.507 x Tot Negative: 0.483
Acc Geral: 0.523

MSE: 171.562 %
MAPE: nan %
Retorno Médio: 8.162 


Portfolio:
BBSE3.SA    0.1
SLCE3.SA    0.1
COGN3.SA    0.1
TIMS3.SA    0.1
WEGE3.SA    0.1
GGBR4.SA    0.1
TOTS3.SA    0.1
EMBR3.SA    0.1
CYRE3.SA    0.1
PRIO3.SA    0.1
Name: 0, dtype: float64

Janela Treino:          227
Janela Teste:         23
Retorno:               -0.010382391124604595
Retorno Looser:               -0.035335584725683655
Retorno Benchmark:     -0.030616023569176032
Retornos Acumulados:   4.261093266944775 x 1.5509309531461188 x 1.205429162019319 x 0.30877438430179033
Volatilidade:          0.22525909140183983
Sharpe Ratio:          -0.6663588258487493
Maximum Draw Down:     -0.05679919462450148
Value at Risk:         -0.01959689737936985

    T   F
P   7  13
N  25  13

Acc Positive: 0.35 x Tot Positive: 0.345
Acc Negative: 0.658 x Tot Negative: 0.655
Acc Geral: 0.552

      T     F
P  1609  1442
N  1391  1300

Acc Positive: 0.527 x Tot Positive: 0.507
Acc Negative: 0.517 x Tot Negative: 0.493
Acc Geral: 0.522

MSE: 171.577 %
MAPE: nan %
Retorno Médio: 6.945 %
Ra

In [ ]:
comp = pd.DataFrame()
comp['Strategy'] = strategy_return
comp['^BVSP'] = bench_return

In [ ]:
cf_analysis(total_confusion_matrix)

In [ ]:
rankings = pd.DataFrame(index=range(len(selected_rank_winners)),\
                                       columns=['Winners', 'Loosers', 'Naive'])

rankings['Winners'] = selected_rank_winners
#comparative['Benchmark'] = bench_return
rankings['Naive'] = [31.5 for i in range(len(selected_rank_winners))]
rankings['Loosers'] = selected_rank_loosers

rankings.plot()

In [ ]:
(rankings['Loosers'] - rankings['Winners']).rolling(12).mean().plot()

In [ ]:
#Montando a planilha de resultados

analysis_sheet = pd.DataFrame(data = strategy_return)
analysis_sheet['Return'] = empyrical.annual_return(returns=strategy_return)
analysis_sheet['Volatility'] = empyrical.annual_volatility(returns=strategy_return)
analysis_sheet['Sharpe Ratio'] = empyrical.sharpe_ratio(returns=strategy_return)
analysis_sheet['Max DD'] = empyrical.max_drawdown(returns=strategy_return)
analysis_sheet['CVaR'] = empyrical.stats.conditional_value_at_risk(returns=strategy_return, cutoff= 0.05)
#analysis_sheet['Beta'] = empyrical.beta(returns=retorno, factor_returns=benchmark_rets['^BVSP'])
analysis_sheet['retorno ac.'] = (strategy_return+1).cumprod()
#analysis_sheet['N ativos médio'] = np.mean(n_ativos)
  
analysis_sheet.to_excel('Resultados/BR/'+ml_method + '-' + opt_method + '-' + \
trainning_test_size + '-' + test_size + '_'+data_source+'.xlsx', index = True)


#%% Graphs
#Drawdown
fig, ax1 = plt.subplots(figsize=(22,16))
drawdown_fig=pf.plot_drawdown_periods(returns=strategy_return, label= 'Drawdown')
drawdown_fig=pf.plot_drawdown_periods(returns=strategy_return)
plt.title('Top 10 Drawdown', fontsize=22)
plt.ylabel('Retorno', fontsize=18)
plt.legend(fontsize=18, loc=2)
drawdown_fig.figure.savefig('Drawdown.png', format = 'png')
    
#Volatility
fig, ax1 = plt.subplots(figsize=(22,16))
rolling_volatility_fig=pf.plot_rolling_volatility(returns=strategy_return,\
                    factor_returns=returns_benchmark['^BVSP'][retorno.index[0]:retorno.index[-1]])
plt.title('Rolling Volatility', fontsize=22)
plt.ylabel('Volatilidade', fontsize=18)
plt.legend(fontsize=18, loc=2)
rolling_volatility_fig.figure.savefig('Rolling Volatility.png', format = 'png')
    
#Cumulative Returns
fig, ax1 = plt.subplots(figsize=(22,16))
plt.plot((1+strategy_return).cumprod(), 'b', label='Portfolio')
plt.plot((1+returns_benchmark['^BVSP'][strategy_return.index[0]:strategy_return.index[-1]]).cumprod(),\
         'r', label='^BVSP')
plt.title('Cumulative Returns', fontsize=22)
plt.ylabel('Retorno', fontsize=18)
plt.legend(fontsize=18, loc=2)
fig.savefig('Retorno Acumulado.png', format = 'png')

In [ ]:
#Não estou conseguindo funcionar com benchmarks, vou deixar comentado

#pf.create_full_tear_sheet(strategy_return, benchmark_rets=bench_return)
#pf.create_full_tear_sheet(strategy_return, benchmark_rets=naive_return)
#pf.create_full_tear_sheet(strategy_return, benchmark_rets=looser_return)
pf.create_full_tear_sheet(strategy_return)

In [ ]:
comparative = pd.DataFrame(index=strategy_return.index, \
                           columns=['Strategy', 'Benchmark', 'Naive', 'Looser'])

comparative['Strategy'] = strategy_return
comparative['Benchmark'] = bench_return
comparative['Naive'] = naive_return
comparative['Looser'] = looser_return

(1+comparative).cumprod().plot()

In [ ]:
(((1+comparative).cumprod()['Strategy']-(1+comparative).cumprod()['Looser'])-1).plot()

In [ ]:
((1+comparative).cumprod()['Strategy']-(1+comparative).cumprod()['Naive']).plot()

In [ ]:
((1+comparative).cumprod()['Strategy']-(1+comparative).cumprod()['Benchmark']).plot()

In [ ]:
sns.heatmap(comparative.corr(), annot=True, cmap='coolwarm')

In [ ]:
comparative['Strategy > Benchmark'] = np.where(comparative['Strategy'] > comparative['Benchmark'], 1, 0)

print('A estratégia superou o benchmark', \
      round((comparative['Strategy > Benchmark'].sum()/len(comparative))*100, 2),'% dos dias')

comparative.drop('Strategy > Benchmark', axis=1, inplace=True)

In [ ]:
sns.distplot(comparative['Strategy'])

In [ ]:
sns.distplot(comparative['Benchmark'])

In [ ]:
sns.distplot(comparative['Naive'])

In [ ]:
sns.boxplot(x = 'Strategy', data = comparative)

In [ ]:
sns.boxplot(x = 'Benchmark', data = comparative)

In [ ]:
sns.boxplot(x = 'Naive', data = comparative)

In [ ]:
sns.pairplot(comparative)

In [ ]:
comparative['Month'] = pd.Series(str)
for i in comparative.index:
    if i.month<10:
        comparative.loc[i,'Month'] = str(i.year)+'-0'+str(i.month)
    else:
        comparative.loc[i,'Month'] = str(i.year)+'-'+str(i.month)
    
comparative

In [ ]:
comparative_month = comparative.copy()

comparative_month.loc[:,'Strategy'] = comparative_month.loc[:,'Strategy'] + 1
comparative_month.loc[:,'Benchmark'] = comparative_month.loc[:,'Benchmark'] + 1
comparative_month.loc[:,'Naive'] = comparative_month.loc[:,'Naive'] + 1

comparative_month = comparative_month.groupby('Month').prod() - 1 
comparative_month

In [ ]:
sns.distplot(comparative_month['Strategy'])

In [ ]:
sns.distplot(comparative_month['Benchmark'])

In [ ]:
sns.distplot(comparative_month['Naive'])

In [ ]:
sns.boxplot(x = 'Strategy', data = comparative_month)

In [ ]:
sns.boxplot(x = 'Benchmark', data = comparative_month)

In [ ]:
sns.boxplot(x = 'Naive', data = comparative_month)

In [ ]:
sns.pairplot(comparative_month)